

# Assignment 3
Github Link [Github](https://github.com/Chang-Liu-Harry/6220DataMining)

# Setting Up Spark in Colabs

## Virtual Machine Setup


In [ ]:
#@title Download and install the PySpark packages

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
#@title Setup Spark and Data

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Friend Recommendation

In [5]:
# Download the txt file
!wget -O soc-LiveJournal1Adj.txt 'https://course.ccs.neu.edu/cs6220/fall2023/homework-3/soc-LiveJournal1Adj.txt'


--2024-02-14 07:50:12--  https://course.ccs.neu.edu/cs6220/fall2023/homework-3/soc-LiveJournal1Adj.txt
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156181 (4.0M) [text/plain]
Saving to: ‘soc-LiveJournal1Adj.txt’

soc-LiveJournal1Adj 100%[===================>]   3.96M  15.3MB/s    in 0.3s    

2024-02-14 07:50:13 (15.3 MB/s) - ‘soc-LiveJournal1Adj.txt’ saved [4156181/4156181]



Convert the txt to rdd and format the rdd

In [6]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

rdd = sc.textFile("soc-LiveJournal1Adj.txt")

def parse_line(line):
    parts = line.split('\t')
    user = int(parts[0])
    friends = parts[1].split(',')
    return (user, [int(friend) for friend in friends if friend.isdigit()])

user_friends = rdd.map(parse_line)

Any pair in the friends list, have a mutual friend already, which is the user.

If we can make this rdd to the pairs of friends, then we can know the count of mutual friends using reduceByKey.

In [7]:
from itertools import combinations

mutual_pair = user_friends.flatMap(lambda x: [(tuple(sorted(pair)), 1) for pair in combinations(x[1], 2)])

But for those friends pairs, we don't know if they are friends already.
If we create another map of those who are already friends, and union the two RDD, we can mark those already friends and filter them out.

In [8]:
actual_friends = user_friends.flatMap(lambda x: [(tuple(sorted([x[0], f])), float('-inf')) for f in x[1]])
all_pairs = mutual_pair.union(actual_friends)
mutual_friend_count = all_pairs.reduceByKey(lambda x,y: x + y).filter(lambda x: x[1] != float('-inf'))

Now we have the count of mutual friends for each user pair.

We now format the RDD to [user, count of mutual friends with other users)

And then rank the list taking top 10

In [12]:
recommendations = mutual_friend_count.flatMap(
    lambda x: [(x[0][0], (x[0][1], x[1])), (x[0][1], (x[0][0], x[1]))]
).groupByKey()
top_recommendations = recommendations.map(
    lambda x: (x[0], sorted(x[1], key=lambda pair: pair[1], reverse=True)[:10])
)

Format the output to satisfy the format requirements:

In [57]:
def format_output(user_recs):
    user, recs = user_recs
    return f"{user}\t{','.join([str(rec[0]) for rec in recs])}"

output = top_recommendations.map(format_output)
output.saveAsTextFile("Assignment3_Chang")

Test if the results are correct:

I first printed out the top 10 but is not exact the same as the result given in our assignment requirements.

Therefore I printed out the top 20 recommendations, to find out there are multiple users with same amount of mutual friends and my algorithm is correct but the result can be different while equally satisfy the requirement of the assignment.

In [11]:
recommendations = mutual_friend_count.flatMap(
    lambda x: [(x[0][0], (x[0][1], x[1])), (x[0][1], (x[0][0], x[1]))]
).groupByKey()
top_recommendations = recommendations.map(
    lambda x: (x[0], sorted(x[1], key=lambda pair: pair[1], reverse=True)[:20])
)

user_ids = [11]

specific_user_recommendations = top_recommendations.filter(lambda x: x[0] in user_ids)

results = specific_user_recommendations.collect()

for user, recs in results:
  print(f"User {user}: {', '.join([str(rec[0]) + '-' + str(rec[1]) for rec in recs])}")

User 11: 27552-4, 27667-3, 27620-3, 33192-3, 32072-3, 27600-3, 27617-3, 27573-3, 7785-3, 27589-3, 27574-3, 27590-3, 27587-2, 27555-2, 27655-2, 27583-2, 35451-2, 25195-2, 10-2, 12-2


Lastly, I am printing out the results for the list of user ids.

Considering it's quite common for recommended users to have same amount of mutual friends, I don't think there is one only correct answer for the top 10.

In [13]:
user_ids = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]

# Filter the final_recommendations for specific user IDs
specific_user_recommendations = top_recommendations.filter(lambda x: x[0] in user_ids)

# Collect the results
results = specific_user_recommendations.collect()

# Print the recommendations
for user, recs in results:
    print(f"User {user}: {','.join([str(rec[0]) for rec in recs])}")

User 9020: 9021,9022,9016,9017,317,9023
User 9992: 9987,9989,35667,9991
User 924: 45881,439,2409,15416,11860,43748,6995
User 9021: 9020,9017,9022,9016,317,9023
User 9993: 9991,13877,34485,37941,13134,13478,34642,34299
User 8941: 8943,8944,8940
User 8942: 8939,8940,8944,8943
User 9022: 9020,9021,9019,9016,9017,9023,317
User 9990: 34299,13134,13478,34642,13877,34485,37941
User 9019: 9022,9023,317
